# Access Synapse SQL table from Synapse Spark

This notebook provides examples of how to read data from Synapse SQL into a Spark context and how to write the output of Spark jobs into an Synapse SQL table.


## Limits
- Scala is the only supported language by the Spark-SQL connector.
- The Spark connector can only read colummns without space in its header in the sql pool.
- Columns with time definition in the sql pool not yet supported.
- You need to define a container on the workspace's primary or linked storage as the temp data folder.

## Pre-requisites
You need to be db_owner to read and write in sql pool. Ask your admin to run the following command with your AAD credential:

    
    EXEC sp_addrolemember 'db_owner', 'AAD@contoso.com'

## Load a sample data

Let's first load the [Public Holidays](https://azure.microsoft.com/en-us/services/open-datasets/catalog/public-holidays/) of last 6 months from Azure Open datasets as a sample.


In [5]:
%%pyspark 
# Load sample data from azure open dataset in pyspark
from azureml.opendatasets import PublicHolidays

from datetime import datetime
from dateutil import parser
from dateutil.relativedelta import relativedelta


end_date = datetime.today()
start_date = datetime.today() - relativedelta(months=6)
hol = PublicHolidays(start_date=start_date, end_date=end_date)
hol_df = hol.to_spark_dataframe()

print('Register the DataFrame as a SQL temporary view: source')
hol_df.createOrReplaceTempView('source')

Register the DataFrame as a SQL temporary view: source

In [6]:
// Remove datetime from the data source
val holiday_nodate = spark.sql("SELECT countryOrRegion, holidayName, normalizeHolidayName,isPaidTimeOff,countryRegionCode FROM source")
holiday_nodate.show(5,truncate = false)

holiday_nodate: org.apache.spark.sql.DataFrame = [countryOrRegion: string, holidayName: string ... 3 more fields]
+---------------+-------------------------+-------------------------+-------------+-----------------+
|countryOrRegion|holidayName              |normalizeHolidayName     |isPaidTimeOff|countryRegionCode|
+---------------+-------------------------+-------------------------+-------------+-----------------+
|Czech          |Den české státnosti      |Den české státnosti      |null         |CZ               |
|Norway         |Søndag                   |Søndag                   |null         |NO               |
|Sweden         |Söndag                   |Söndag                   |null         |SE               |
|India          |Gandhi Jayanti           |Gandhi Jayanti           |true         |IN               |
|Germany        |Tag der Deutschen Einheit|Tag der Deutschen Einheit|null         |DE               |
+---------------+-------------------------+-------------------------+-

## Write a Spark dataframe into your sql pool


In [8]:
// Write the dataframe into your sql pool
import org.apache.spark.sql.SqlAnalyticsConnector._
import com.microsoft.spark.sqlanalytics.utils.Constants

val sql_pool_name = "Your sql pool name" //fill in your sql pool name

holiday_nodate.write
    .sqlanalytics(s"$sql_pool_name.dbo.PublicHoliday", Constants.INTERNAL)


import org.apache.spark.sql.SqlAnalyticsConnector._
import com.microsoft.spark.sqlanalytics.utils.Constants
sql_pool_name: String = mysqlpool
account_name: String = ruxune
temp_folder: String = data

Now open Synapse object explorer and go to **Data**->**Databases**->**<your sql pool name>**->**Tables**, you will see the new **dbo.PublicHoliday** table show up there.

## Read from a SQL Pool table with Spark



In [10]:
// Read  the table we just created in the sql pool as a Spark dataframe
val spark_read = spark.read.
    sqlanalytics(s"$sql_pool_name.dbo.PublicHoliday")
spark_read.show(5, truncate = false)

spark_read: org.apache.spark.sql.DataFrame = [countryOrRegion: string, holidayName: string ... 3 more fields]
+---------------+-------------------------+-------------------------+-------------+-----------------+
|countryOrRegion|holidayName              |normalizeHolidayName     |isPaidTimeOff|countryRegionCode|
+---------------+-------------------------+-------------------------+-------------+-----------------+
|Czech          |Den české státnosti      |Den české státnosti      |null         |CZ               |
|Norway         |Søndag                   |Søndag                   |null         |NO               |
|Sweden         |Söndag                   |Söndag                   |null         |SE               |
|India          |Gandhi Jayanti           |Gandhi Jayanti           |true         |IN               |
|Germany        |Tag der Deutschen Einheit|Tag der Deutschen Einheit|null         |DE               |
+---------------+-------------------------+-------------------------+-----